In [ ]:
# from google.colab import drive
# import os
# drive.mount('/content/drive')
# os.chdir("drive/MyDrive/Bootcamp_Henry/pro_final")
# print(os.getcwd())

Mounted at /content/drive
/content/drive/MyDrive/Bootcamp_Henry/pro_final


# **Objetivo del Cuaderno Jupyter**
- Ya disponemos de las tablas que se muestran en la imagen en nuestro `Datawarehouse` ahora debemos a partir de las mismas crear las tablas `Desnormalizadas` que se usarán en el proceso de `Machinne Learning` para crear el `Modelo de recomendación`.
- Tablas a crear:
    - `ml_user`
    - `ml_reviews`
    - `ml_business`
<img src="0_TABLAS_YELP.png">
<br>
<br>
# **NOTA:**<br>
- El Proceso que se muestra en este cuaderno se corrió en Local.
- Este proceso, servirá de guía para que nuestro equipo de Ingeniería replique dichas transformaciones en la Nube, haciendo los ajustes necesarios de sintaxis y configuración
- Este proceso se ha dividido en Funciones, por recomendación de nuestro Tutor, quien nos explicó las ventajas a la hora de `desplegar` y `debuguear` en la nube sí presentamos el código estructurado en funciones.

In [2]:
import pandas as pd
from textblob import TextBlob

In [3]:
# Necesario para hacer `ml_reviews`
reviews = pd.read_parquet('datasets/yelp/filtrados_segun_maestro_cat/y_reviews.parquet')

# Necesario para hacer `ml_business`
attributes = pd.read_parquet('datasets/yelp/filtrados_segun_maestro_cat/y_attributes_business.parquet')
tip = pd.read_parquet('datasets/yelp/filtrados_segun_maestro_cat/y_tip.parquet')
business = pd.read_parquet('datasets/yelp/filtrados_segun_maestro_cat/y_business.parquet')
checkin = pd.read_parquet('datasets/yelp/filtrados_segun_maestro_cat/y_checkin.parquet')

# Necesario para hacer `ml_users`
user = pd.read_parquet('datasets/yelp/filtrados_segun_maestro_cat/y_user.parquet')

***
## Funciones para crear `ml_business`

In [ ]:
def retorna_lista_atributos(attributes:pd.DataFrame) -> tuple[list,list]:
  """
    retorna_lista_atributos: retorna la lista de Atributos en primer nivel de anidamiento
                             y las listas anidadas en 2do Nivel de Anidamiento
    Args:
        attributes (pl.DataFrame): dataframe que contiene la tabla `y_attributes_business` de la imagen

    Returns:
        tuple[list,list]: Listas tanto de atributos en primer y segundo nivel de anidamiento
    """
  set_attributes = set()

  #Cuantifico cuantos Atributos en Primer Nivel de Anidamiento hay
  for i in range(len(attributes)):
      diccionario = attributes.iloc[i,1]
      if diccionario:
          for key in diccionario.keys():
              set_attributes.add(key)

  list_attributes = list(set_attributes)

  #'HairSpecializesIn'
  attributes_hair_specializes = ['straightperms', 'coloring', 'extensions', 'africanamerican',
                                'curly', 'kids', 'perms', 'asian']

  #'DietaryRestrictions'
  attributes_dietary_restrictions = ['dairy-free', 'gluten-free', 'vegan', 'kosher', 'halal', 'soy-free', 'vegetarian']

  #'BestNights'
  attributes_bestnights = ['monday', 'tuesday', 'friday', 'wednesday', 'thursday', 'sunday', 'saturday']

  #Ambience
  attributes_ambience = ['romantic', 'intimate', 'touristy', 'hipster', 'divey',
                        'classy', 'trendy', 'upscale', 'casual']
  #Music
  attributes_music = ['dj', 'background_music', 'no_music', 'jukebox', 'live', 'video', 'karaoke']

  #'BusinessParking'
  attributes_business_parking = ['garage', 'street', 'validated', 'lot', 'valet']

  #'GoodForMeal'
  attributes_good_for_meal = ['dessert', 'latenight', 'lunch', 'dinner', 'brunch', 'breakfast']

  #Atributos que tienen un segundo nivel de Anidamiento
  nested_attributes = ['BusinessParking', 'Ambience', 'GoodForMeal', 'BestNights', 'Music',
                      'DietaryRestrictions', 'HairSpecializesIn']

  matrix_nested_attibutes = [attributes_hair_specializes, attributes_dietary_restrictions,
                            attributes_bestnights, attributes_ambience, attributes_music,
                            attributes_business_parking, attributes_good_for_meal, nested_attributes]

  #Aplico lógica de recorrido matricial para obtener todos los atributos en una lista (ojo Teoricamente esto NO es una matriz)
  for i in range(len(matrix_nested_attibutes)):
      for j in range(len(matrix_nested_attibutes[i])):
          list_attributes.append(matrix_nested_attibutes[i][j])

  # Elimino los atributos que tienen 2do nivel de anidamineto (ya que inclui sus atributos hijos)
  for element in nested_attributes:
      list_attributes.remove(element)

  print(f"Numero de Atributos sin el segundo nivel de anidamiento --> {len(set_attributes)}")

  print(f"Numero de Atributos Desanidados los 2 niveles de anidamiento--> {len(list_attributes)}")
  return (list_attributes, nested_attributes)

In [ ]:
def retorna_df_total_attributes(attributes:pd.DataFrame, list_attributes:list, nested_attributes:list) -> pd.DataFrame:
  """
    retorna_df_total_attributes: Crea un dataframe, donde por cada `business_id` habran una columna por cada
                                 atributo, HORIZONTALIZO la data, con la intención de luego hacer un Join con
                                 la tabla `y_business` Nota: Los atributos en los cuales los Negocios de esta
                                 data NINGUNO lo dispone (es decir todos los registros tienen 0 en esa columna)
                                 automaticamente los borro, ya que no van a aportar diferenciación para el modelo  

    Args:
        attributes (pd.DataFrame): dataframe que contiene la tabla `y_attributes_business` de la imagen
        list_attributes (list): Lista de Atributos en Primer nivel de Anidamiento
        nested_attributes (list): Lista con las Listas que tienen 2do nivel de Anidamiento

    Returns:
        pd.DataFrame: Dataframe de Atributos totalmente desanidados
    """
  rows = []
  for i in range(len(attributes)):
      diccionario = attributes.iloc[i,1]
      if diccionario:
          # Inicializo el Diccionario en cada Iteracion
          diccionario_row = {'business_id': attributes.iloc[i,0]}
          for attribute in list_attributes:
              diccionario_row[attribute] = 0

          for key,value in diccionario.items():
              # Entra en if si no tiene 2do nivel de anidamiento
              if key not in nested_attributes:
                  verdadero = 1 if value else 0
                  diccionario_row[key] = verdadero
              else:
                  if type(key) == dict:
                      for key_2, value_2 in eval(diccionario[key]).items():
                          verdadero = 1 if value_2 else 0
                          diccionario_row[key_2] = verdadero
          rows.append(diccionario_row)
  df_total_attributes = pd.DataFrame(rows)
  #Localizando Columnas que no agregan capacidad de diferenciacion para el modelo
  columnas_a_eliminar = []
  for column in df_total_attributes.columns[1:]:
      if df_total_attributes[column].sum() == 0:
          columnas_a_eliminar.append(column)
  #Elimino las columnas que no agregan valor a la diferenciacion
  df_total_attributes = df_total_attributes.drop(columns=columnas_a_eliminar)
  return df_total_attributes

In [ ]:
def business_merge_attributes(business:pd.DataFrame, df_total_attributes:pd.DataFrame, estados:list) -> pd.DataFrame:
  """
    business_merge_attributes: Realiza un Join entre el dataframe creado de atributos y el dataframe que representa 
                              la tabla `y_business` de la Imagen.  

    Args:
        business (pd.DataFrame): Dataframe que contiene la data de la tabla `y_business`
        df_total_attributes (pd.DataFrame): Dataframe que contiene la data de los atributos totalmente desanidados
        estados (list): Lista de Estados para realizar un filtro de la data

    Returns:
        pd.DataFrame: Dataframe resultante de todos los cambios dentro de la función
    """
  mask = business['business_id'].isin(df_total_attributes['business_id'].unique().tolist())
  business = business[mask]
  business = business.reset_index(drop=True)

  #Como se han manipulado los datos para que business y df_total_attributes tengan las mismas filas
  business = business.merge(df_total_attributes, on='business_id', how='inner')

  # ACA el filtrado por estados
  mask = business['state'].isin(estados)
  business = business[mask]
  business = business.reset_index(drop=True)

  def retorna_apertura(horario:str) -> float:
    if horario == 'No Disponible':
      return 0
    return horario.split('-')[0]

  def retorna_cierre(horario:str):
    if horario == 'No Disponible':
      return 0
    return horario.split('-')[1]

  days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
  days_open = ['monday_open', 'tuesday_open', 'wednesday_open', 'thursday_open', 'friday_open', 'saturday_open', 'sunday_open']
  days_close = ['monday_dura', 'tuesday_dura', 'wednesday_dura', 'thursday_dura', 'friday_dura', 'saturday_dura', 'sunday_dura']

  for i in range(7):
    business[days_open[i]] = business[days[i]].apply(lambda x: retorna_apertura(x))
    business[days_close[i]] = business[days[i]].apply(lambda x: retorna_cierre(x))

  business = business.drop(columns=days)
  business = business.reset_index(drop=True)
  return business


In [ ]:
def retorna_tip_totalizados(tip:pd.DataFrame, lista_business_id:list) -> pd.DataFrame:
  """
    retorna_tip_totalizados: Aplica analisis de sentimientos a la columna `text` de la tabla `y_tip`
                             y guarda estos resultados que van de -1 a 1 en la columna polaridad_tip
                             luego aplicacion de funciones de agregacion (average o suma, según sea el caso)
                             para cada `business_id`

    Args:
        tip (pd.DataFrame): Dataframe que contiene la data de la tabla `y_tip` de la Imagen
        lista_business_id (list): Lista de Negocios donde se realizará el procedimiento de ML

    Returns:
        pd.DataFrame: Dataframe resultante de las transformaciones en esta función
    """
  mask = tip['business_id'].isin(lista_business_id)
  tip = tip[mask]
  tip = tip.reset_index(drop=True)

  tip['polaridad_tip'] = tip['text'].apply(lambda x: TextBlob(x).sentiment.polarity)

  lista_business_id_tip = tip['business_id'].unique().tolist()

  data_tip = []
  for i in range(len(lista_business_id_tip)):
      row = {}
      row['business_id'] = lista_business_id_tip[i]
      row['tiene_tip'] = 1
      mask = tip['business_id'] == lista_business_id_tip[i]
      # Saco el promedio de los Sentimientos
      polaridad_tip = tip[mask]['polaridad_tip'].mean()
      # Saco la sumatoria de los cumplidos
      compliment_count = tip[mask]['compliment_count'].sum()
      row['polaridad_tip'] = polaridad_tip
      row['compliment_count'] = compliment_count
      data_tip.append(row)
  df_tip = pd.DataFrame(data_tip)
  return df_tip

In [ ]:
def retorna_checkin_totalizados(checkin:pd.DataFrame, lista_business_id:list) -> pd.DataFrame:
  """
    retorna_checkin_totalizados: aplicacion de funciones de agregacion (suma) para cada `business_id`

    Args:
        checkin (pd.DataFrame): Dataframe que contiene la data de la tabla `y_checkin` de la Imagen
        lista_business_id (list): Lista de Negocios donde se realizará el procedimiento de ML

    Returns:
        pd.DataFrame: Dataframe resultante de las transformaciones en esta función
    """
  mask = checkin['business_id'].isin(lista_business_id)
  checkin = checkin[mask]
  checkin = checkin.reset_index(drop=True)

  lista_business_id_checkin = checkin['business_id'].unique().tolist()

  data_checkin = []
  for i in range(len(lista_business_id_checkin)):
    row = {}
    mask = checkin['business_id'] == lista_business_id_checkin[i]
    row['business_id'] = lista_business_id_checkin[i]
    row['tiene_checkin'] = 1
    row['cantidad_checkins'] = checkin[mask]['date'].count()
    data_checkin.append(row)
  df_checkin = pd.DataFrame(data_checkin)
  return df_checkin

In [ ]:
def retorna_ml_business(business:pd.DataFrame, df_tip:pd.DataFrame, df_checkin:pd.DataFrame) -> pd.DataFrame:
  """
    retorna_ml_business: Recibe los Dataframes creados en las funciones `business_join_attributes`, 
                         `retorna_tip_totalizados` y `retorna_checkin_totalizados` y crea el Dataframe 
                         será guardado como `ml_business` 

    Args:
        business (pd.DataFrame): Dataframe resultante de la función `business_join_attributes`
        df_tip (pd.DataFrame): Dataframe resultante de la función `retorna_tip_totalizados`
        df_checkin (pd.DataFrame): Dataframe resultante de la función `retorna_checkin_totalizados`

    Returns:
        pd.DataFrame: Dataframe que tendrá la data de `ml_business`
    """
  ml_business = business.merge(df_tip, on='business_id', how='left')
  ml_business = ml_business.merge(df_checkin, on='business_id', how='left')

  columnas_a_rellenar = ['tiene_tip', 'polaridad_tip', 'compliment_count', 'tiene_checkin', 'cantidad_checkins']
  for column in columnas_a_rellenar:
    ml_business[column] = ml_business[column].fillna(0)
  return ml_business

***
## Pasos para Crear la tabla `ml_business`

In [ ]:
# Necesito los Atributos en 1er y 2do Nivel de Anidamiento
list_attributes, nested_attributes = retorna_lista_atributos(attributes=attributes)

Numero de Atributos sin el segundo nivel de anidamiento --> 38
Numero de Atributos Desanidados los 2 niveles de anidamiento--> 87


In [ ]:
# Dataframe con los `business_id` y TODOS los Atributos desanidados en forma de Dumies
df_total_attributes = retorna_df_total_attributes(attributes=attributes, list_attributes=list_attributes, nested_attributes=nested_attributes)

In [ ]:
estados = ['PA', 'TN', 'CA', 'IN', 'MO', 'FL', 'LA', 'NV', 'ID', 'AZ', 'IL', 'DE']
# Haciendo el Join de bussines y Atributos, y en el Filtrado por estado dejo por fuera a los que tienen
# pocos locales, como por ejemplo Texas que tiene un solo local (en esta data de muestra)
business = business_merge_attributes(business=business, df_total_attributes=df_total_attributes, estados=estados)

del df_total_attributes

In [ ]:
lista_business_id = business['business_id'].unique().tolist()

# Analisis de sentimiento (columna `polaridad_tip`) y hago promedio por business_id
df_tip = retorna_tip_totalizados(tip=tip, lista_business_id=lista_business_id)

# Sumarizo checkines por cada business_id
df_checkin = retorna_checkin_totalizados(checkin=checkin, lista_business_id=lista_business_id)

# Hago join de todas las tablas relacionadas con business
ml_business = retorna_ml_business(business=business, df_tip=df_tip, df_checkin=df_checkin)

Guardamos la Tabla `ml_business`

In [ ]:
ml_business.to_parquet('datasets/yelp/ml_tables/ml_business.parquet', index=False)

***
## Pasos para crear la tabla `ml_reviews`

Haciendo un pequeño filtrado en reviews y Guardando la tabla

In [14]:
mask = reviews['business_id'].isin(ml_business['business_id'].unique().tolist())
reviews = reviews[mask]
columnas_a_borrar = ['date', 'text', 'useful', 'funny', 'cool']
reviews = reviews.drop(columns=columnas_a_borrar)
reviews = reviews.reset_index(drop=True)

Guardamos la tabla `ml_reviews`

In [ ]:
reviews.to_parquet('datasets/yelp/big_tables/ml_reviews.parquet', index=False)

***
## Pasos para crear la tabla `ml_users`

In [15]:
lista_usuarios = reviews['user_id'].unique().tolist()
columnas = ['user_id', 'name','average_stars']
mask = user['user_id'].isin(lista_usuarios)
user = user[mask]
user = user.reset_index(drop=True)

Guardando la Tabla

In [ ]:
user.to_parquet('datasets/yelp/ml_tables/ml_user.parquet', index=False)